# word2vec: How To Prep Word Vectors For Modeling

### Train Our Own Model

In [15]:
# Read in the data, clean it, split it into train and test sets, and then train a word2vec model
import gensim
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
pd.set_option('display.max_colwidth', 100)

messages = pd.read_csv('../../../data/spam.csv', encoding='latin-1')
messages = messages.drop(labels = ["Unnamed: 2", "Unnamed: 3", "Unnamed: 4"], axis = 1)
messages.columns = ["label", "text"]


messages['text_clean'] = messages['text'].apply(lambda x: gensim.utils.simple_preprocess(x))
X_train, X_test, y_train, y_test = train_test_split(messages['text_clean'],
                                                    messages['label'], test_size=0.2)

w2v_model = gensim.models.Word2Vec(X_train,
                                   vector_size=100,
                                   window=5,
                                   min_count=2)

### Prep Word Vectors

In [16]:
# Generate a list of words the word2vec model learned word vectors for
w2v_model.wv.index_to_key

['to',
 'you',
 'the',
 'and',
 'in',
 'is',
 'me',
 'my',
 'it',
 'for',
 'your',
 'of',
 'call',
 'that',
 'have',
 'on',
 'are',
 'so',
 'now',
 'can',
 'but',
 'not',
 'or',
 'we',
 'do',
 'be',
 'at',
 'will',
 'if',
 'ur',
 'no',
 'with',
 'get',
 'just',
 'gt',
 'this',
 'lt',
 'how',
 'go',
 'ok',
 'up',
 'what',
 'from',
 'when',
 'all',
 'out',
 'll',
 'then',
 'know',
 'free',
 'like',
 'good',
 'am',
 'he',
 'day',
 'there',
 'come',
 'was',
 'got',
 'its',
 'time',
 'only',
 'love',
 'send',
 'want',
 'text',
 'one',
 'txt',
 'as',
 'she',
 'going',
 'today',
 'don',
 'by',
 'home',
 'need',
 'see',
 'lor',
 'still',
 'sorry',
 'stop',
 'our',
 'back',
 'about',
 'reply',
 'take',
 'they',
 'think',
 'dont',
 'tell',
 'later',
 'week',
 'mobile',
 'new',
 'pls',
 'hi',
 'her',
 'da',
 'any',
 'please',
 'some',
 'did',
 'here',
 'oh',
 'phone',
 'ì_',
 'been',
 're',
 'well',
 'wat',
 'night',
 'an',
 'much',
 'where',
 'has',
 'who',
 'great',
 'dear',
 'hope',
 'hey',
 '

The following code does not work because it would create an np.array where each line has a different shape. Therefore we split this into 2 steps. 
- first we create a list with the vectors with differents lengths 
- second we create an average documents and add it. 
- one special remark: as there are some texts where no word is occuring in any other document, there are empty vectors in the list. For those we need to create a np.zeros(100) array and add it

In [17]:
# Generate aggregated sentence vectors based on the word vectors for each word in the sentence
# w2v_vect = np.array([np.array([w2v_model.wv[i] for i in ls if i in w2v_model.wv.index_to_key])
#                      for ls in X_test])

In [18]:

w2v_vec_list = [np.array([w2v_model.wv[i] for i in ls if i in w2v_model.wv.index_to_key for ls in X_test])]

In [19]:
w2v_vec_list

[array([[-0.3477871 ,  0.5561692 ,  0.1495496 , ..., -0.5749841 ,
          0.11042677, -0.11492893],
        [-0.3477871 ,  0.5561692 ,  0.1495496 , ..., -0.5749841 ,
          0.11042677, -0.11492893],
        [-0.3477871 ,  0.5561692 ,  0.1495496 , ..., -0.5749841 ,
          0.11042677, -0.11492893],
        ...,
        [-0.26958326,  0.43677616,  0.12404623, ..., -0.42899778,
          0.0998006 , -0.09894712],
        [-0.26958326,  0.43677616,  0.12404623, ..., -0.42899778,
          0.0998006 , -0.09894712],
        [-0.26958326,  0.43677616,  0.12404623, ..., -0.42899778,
          0.0998006 , -0.09894712]], dtype=float32)]

In [20]:
# Why is the length of the sentence different than the length of the sentence vector?
for i, v in enumerate(w2v_vec_list):
    print(len(X_test.iloc[i]), len(v))

20 16725


In [21]:
# Compute sentence vectors by averaging the word vectors for the words contained in the sentence
w2v_vect_avg = []

for vect in w2v_vec_list:
    if len(vect)!=0:
        w2v_vect_avg.append(vect.mean(axis=0))
    else:
        w2v_vect_avg.append(np.zeros(100))

In [23]:
# Are our sentence vector lengths consistent?
for i, v in enumerate(w2v_vec_list):
    print(len(X_test.iloc[i]), len(v))

20 16725
